In [1]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient (no duplication / multi-view)
- Early stop if P & R >= 0.90 on validation
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 224
EPOCHS_CV = 120
BATCH_CV = 22
LR_CV = 0.00022
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_tongue/tongue_1_hb_less_than_9_0/tongue_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "tongue_1_eye_original_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING
# =========================
def load_images(folder, label):
    imgs, lbls = [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
    return imgs, lbls

train_imgs, train_lbls = [], []
val_imgs, val_lbls = [], []
test_imgs, test_lbls = [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l = load_images(folder,label)
    train_imgs+=i; train_lbls+=l
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l = load_images(folder,label)
    val_imgs+=i; val_lbls+=l
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l = load_images(folder,label)
    test_imgs+=i; test_lbls+=l

print(f"TRAIN: {sum(train_lbls)} anemic / {len(train_lbls)} total")
print(f"VAL: {sum(val_lbls)} anemic / {len(val_lbls)} total")
print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS
# =========================
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    preds, probs, labels_all = [], [], []
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        preds.extend((p>0.5).astype(int))
        probs.extend(p)
        labels_all.extend(labels.cpu().numpy().flatten())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn

# =========================
# TRAINING LOOP (FIXED: CV uses ONLY train data)
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # 🔥 CRITICAL FIX: Use ONLY original training data for CV
    X = train_imgs  # ← Only training images
    y = train_lbls  # ← Only training labels
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        # Create train/val splits from ORIGINAL TRAINING DATA ONLY
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            # Print loss every 20 epochs
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            # Early stopping check (every epoch for safety)
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _ = evaluate(model, vl_loader)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        # Final evaluation on fold's validation set
        val_metrics = evaluate(model, vl_loader)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    # Save CV results

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    # Select best fold
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # =========================
    # FINAL EVALUATION ON ORIGINAL TEST SET
    # =========================


    # =========================
    # FINAL EVALUATION ON ORIGINAL TEST SET
    # =========================
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    # Load best model (fix PyTorch 2.6 loading)
    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False  # <--- add this if using PyTorch >=2.6
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    # Evaluate on ORIGINAL TEST SET
    test_metrics = evaluate(model, test_loader)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    return model
'''
# =========================
# RUN
# =========================
if __name__ == "__main__":
    train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
'''
''' single right eye 3 alone model results'''

# =========================
# TFLITE CONVERSION
# =========================

def convert_to_tflite(model, output_dir, resolution):
    """
    Converts a trained PyTorch model to TFLite format via ONNX and TensorFlow.
    """
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    
    # Suppress ONNX-TF/TF warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # --- 1. Preparation and Tracing ---
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # --- 2. PyTorch to ONNX ---
    print("1. Converting PyTorch model to ONNX...")
    try:
        # Export the model
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            dynamic_axes={'input': {0: 'batch_size'}}
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
        
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # --- 3. ONNX to TensorFlow SavedModel ---
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        # Load the ONNX model
        onnx_model = onnx.load(onnx_path)
        
        # Prepare the TensorFlow backend
        tf_rep = prepare(onnx_model)
        
        # Export the model to TensorFlow SavedModel format
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
        
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow SavedModel failed. Make sure onnx-tf and tensorflow are installed. Error: {e}")
        return

    # --- 4. TensorFlow SavedModel to TFLite ---
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        # Initialize the TFLite converter
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        
        # Convert the model
        tflite_model = converter.convert()
        
        # Save the TFLite model
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
        
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# RUN (Update)
# =========================
if __name__ == "__main__":
    # The trained model 'zz' is returned by train_and_eval_single()
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # ⭐ ADDED STEP: Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TRAIN: 261 anemic / 2214 total
VAL: 42 anemic / 369 total
TEST: 47 anemic / 369 total

--- Fold 1 ---
Epoch 20/120 Loss: 17.4395
Epoch 40/120 Loss: 3.9500
Epoch 60/120 Loss: 2.6751
Epoch 80/120 Loss: 2.7094
Epoch 100/120 Loss: 1.6504
Epoch 120/120 Loss: 0.6012
Fold 1 → P=0.273, R=0.113

--- Fold 2 ---
Epoch 20/120 Loss: 13.5626
Epoch 40/120 Loss: 2.6841
Epoch 60/120 Loss: 2.7825
Epoch 80/120 Loss: 1.0533
Epoch 100/120 Loss: 1.5211
Epoch 120/120 Loss: 1.4311
Fold 2 → P=0.286, R=0.154

--- Fold 3 ---
Epoch 20/120 Loss: 17.4963
Epoch 40/120 Loss: 2.7392
Epoch 60/120 Loss: 2.2847
Epoch 80/120 Loss: 1.2580
Epoch 100/120 Loss: 0.9016
Epoch 120/120 Loss: 1.2250
Fold 3 → P=0.235, R=0.154

--- Fold 4 ---
Epoch 20/120 Loss: 13.0562
Epoch 40/120 Loss: 3.5227
Epoch 60/120 Loss: 1.4606
Epoch 80/120 Loss: 1.8726
Epoch 100/120 Loss: 0.5412
Epoch 120/120 Loss: 0.4747
Fold 4 → P=0.500, R=0.115

--- Fold 5 ---
Epoch 20/120 Loss: 13.0257
Epoch 40/120 Loss: 2.

2025-10-13 21:17:22.436731: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-13 21:17:22.438028: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-13 21:17:22.464379: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-13 21:17:22.990843: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to ONNX...
   ✅ ONNX model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_1_eye_original_repro/model.onnx
2. Converting ONNX model to TensorFlow SavedModel...


2025-10-13 21:17:24.692521: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-13 21:17:24.694151: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_20_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will 

INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_1_eye_original_repro/tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_1_eye_original_repro/tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_1_eye_original_repro/tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_1_eye_original_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-10-13 21:17:29.642029: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-10-13 21:17:29.642061: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-10-13 21:17:29.644448: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_1_eye_original_repro/tf_model
2025-10-13 21:17:29.675829: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-10-13 21:17:29.675852: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_1_eye_original_repro/tf_model
2025-10-13 21:17:29.707957: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-10-13 21:17:29.708693: I tensorflow/cc/saved

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_1_eye_original_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.


In [2]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient (no duplication / multi-view)
- Early stop if P & R >= 0.90 on validation
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 224
EPOCHS_CV = 120
BATCH_CV = 22
LR_CV = 0.00022
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_tongue/tongue_2_hb_less_than_9_0/tongue_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "tongue_2_eye_original_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING
# =========================
def load_images(folder, label):
    imgs, lbls = [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
    return imgs, lbls

train_imgs, train_lbls = [], []
val_imgs, val_lbls = [], []
test_imgs, test_lbls = [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l = load_images(folder,label)
    train_imgs+=i; train_lbls+=l
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l = load_images(folder,label)
    val_imgs+=i; val_lbls+=l
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l = load_images(folder,label)
    test_imgs+=i; test_lbls+=l

print(f"TRAIN: {sum(train_lbls)} anemic / {len(train_lbls)} total")
print(f"VAL: {sum(val_lbls)} anemic / {len(val_lbls)} total")
print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS
# =========================
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    preds, probs, labels_all = [], [], []
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        preds.extend((p>0.5).astype(int))
        probs.extend(p)
        labels_all.extend(labels.cpu().numpy().flatten())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn

# =========================
# TRAINING LOOP (FIXED: CV uses ONLY train data)
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # 🔥 CRITICAL FIX: Use ONLY original training data for CV
    X = train_imgs  # ← Only training images
    y = train_lbls  # ← Only training labels
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        # Create train/val splits from ORIGINAL TRAINING DATA ONLY
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            # Print loss every 20 epochs
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            # Early stopping check (every epoch for safety)
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _ = evaluate(model, vl_loader)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        # Final evaluation on fold's validation set
        val_metrics = evaluate(model, vl_loader)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    # Save CV results

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    # Select best fold
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # =========================
    # FINAL EVALUATION ON ORIGINAL TEST SET
    # =========================


    # =========================
    # FINAL EVALUATION ON ORIGINAL TEST SET
    # =========================
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    # Load best model (fix PyTorch 2.6 loading)
    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False  # <--- add this if using PyTorch >=2.6
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    # Evaluate on ORIGINAL TEST SET
    test_metrics = evaluate(model, test_loader)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    return model
'''
# =========================
# RUN
# =========================
if __name__ == "__main__":
    train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
'''
''' single right eye 3 alone model results'''

# =========================
# TFLITE CONVERSION
# =========================

def convert_to_tflite(model, output_dir, resolution):
    """
    Converts a trained PyTorch model to TFLite format via ONNX and TensorFlow.
    """
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    
    # Suppress ONNX-TF/TF warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # --- 1. Preparation and Tracing ---
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # --- 2. PyTorch to ONNX ---
    print("1. Converting PyTorch model to ONNX...")
    try:
        # Export the model
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            dynamic_axes={'input': {0: 'batch_size'}}
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
        
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # --- 3. ONNX to TensorFlow SavedModel ---
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        # Load the ONNX model
        onnx_model = onnx.load(onnx_path)
        
        # Prepare the TensorFlow backend
        tf_rep = prepare(onnx_model)
        
        # Export the model to TensorFlow SavedModel format
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
        
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow SavedModel failed. Make sure onnx-tf and tensorflow are installed. Error: {e}")
        return

    # --- 4. TensorFlow SavedModel to TFLite ---
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        # Initialize the TFLite converter
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        
        # Convert the model
        tflite_model = converter.convert()
        
        # Save the TFLite model
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
        
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# RUN (Update)
# =========================
if __name__ == "__main__":
    # The trained model 'zz' is returned by train_and_eval_single()
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # ⭐ ADDED STEP: Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    

Using device: cuda
GPU: NVIDIA H100 80GB HBM3
TRAIN: 259 anemic / 2204 total
VAL: 43 anemic / 368 total
TEST: 47 anemic / 366 total

--- Fold 1 ---
Epoch 20/120 Loss: 13.9981
Epoch 40/120 Loss: 3.2932
Epoch 60/120 Loss: 1.1374
Epoch 80/120 Loss: 3.9157
Epoch 100/120 Loss: 0.6025
Epoch 120/120 Loss: 2.0979
Fold 1 → P=0.077, R=0.019

--- Fold 2 ---
Epoch 20/120 Loss: 17.9273
Epoch 40/120 Loss: 5.2430
Epoch 60/120 Loss: 2.3776
Epoch 80/120 Loss: 2.9448
Epoch 100/120 Loss: 1.2343
Epoch 120/120 Loss: 1.6316
Fold 2 → P=0.286, R=0.077

--- Fold 3 ---
Epoch 20/120 Loss: 14.7595
Epoch 40/120 Loss: 2.4022
Epoch 60/120 Loss: 1.3159
Epoch 80/120 Loss: 3.1686
Epoch 100/120 Loss: 1.3249
Epoch 120/120 Loss: 1.5639
Fold 3 → P=0.071, R=0.038

--- Fold 4 ---
Epoch 20/120 Loss: 15.4388
Epoch 40/120 Loss: 2.4653
Epoch 60/120 Loss: 2.2577
Epoch 80/120 Loss: 1.4330
Epoch 100/120 Loss: 1.8825
Epoch 120/120 Loss: 1.4303
Fold 4 → P=0.267, R=0.077

--- Fold 5 ---
Epoch 20/120 Loss: 16.2593
Epoch 40/120 Loss: 4.

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_62_x, add_20_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_2_eye_original_repro/tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_2_eye_original_repro/tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_2_eye_original_repro/tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_2_eye_original_repro/tf_model
3. Converting TensorFlow SavedModel to TFLite...


2025-10-13 21:34:08.129686: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-10-13 21:34:08.129722: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-10-13 21:34:08.131926: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_2_eye_original_repro/tf_model
2025-10-13 21:34:08.163427: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-10-13 21:34:08.163445: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_2_eye_original_repro/tf_model
2025-10-13 21:34:08.194695: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-10-13 21:34:08.262147: I tensorflow/cc/saved_model/loader.cc:215] Running ini

   ✅ TFLite model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tongue_2_eye_original_repro/single_eye_resnet18.tflite
   TFLite Model Size: 42.64 MB
✅ TFLite conversion pipeline complete.


In [ ]:
# -*- coding: utf-8 -*-
"""
Reproducible Single-Eye ResNet18 Training (5-fold CV -> select best fold -> TEST)
-------------------------------------------------------------------------------
- FIXED: No data leakage (CV uses only original TRAIN data)
- Strict determinism: fixed seeds, cuDNN deterministic, no TF32, single-thread OpenCV
- Single image per patient (no duplication / multi-view)
- Early stop if P & R >= 0.90 on validation
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from torch.cuda.amp import GradScaler, autocast
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0           # safest for reproducibility
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTION = 780
EPOCHS_CV = 500
BATCH_CV = 8
LR_CV = 0.0003
EARLY_STOP_PR = 0.90      # stop training if P & R >= 0.90
BASE_PATH = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
DATA_DIR = "tri_tongue/tongue_3_hb_less_than_9_0/tongue_extracted/"
OUTPUT_DIR = os.path.join(BASE_PATH, "tongue_3_eye_original_repro")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# =========================
# DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

os.environ["PYTHONHASHSEED"] = str(SEED)
try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
dirs = {
    'anemic_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_train_roi/"),
    'non_train': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_train_roi/"),
    'anemic_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_val_roi/"),
    'non_val': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_val_roi/"),
    'anemic_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_test_roi/"),
    'non_test': os.path.join(BASE_PATH, DATA_DIR, "anemic_not_test_roi/")
}

# =========================
# DATA LOADING
# =========================
def load_images(folder, label):
    imgs, lbls = [], []
    if os.path.exists(folder):
        for f in sorted(os.listdir(folder)):
            if f.endswith(".png"):
                im = cv2.imread(os.path.join(folder, f))
                if im is not None:
                    imgs.append(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                    lbls.append(label)
    return imgs, lbls

train_imgs, train_lbls = [], []
val_imgs, val_lbls = [], []
test_imgs, test_lbls = [], []

for folder, label in [(dirs['anemic_train'],1),(dirs['non_train'],0)]:
    i,l = load_images(folder,label)
    train_imgs+=i; train_lbls+=l
for folder, label in [(dirs['anemic_val'],1),(dirs['non_val'],0)]:
    i,l = load_images(folder,label)
    val_imgs+=i; val_lbls+=l
for folder, label in [(dirs['anemic_test'],1),(dirs['non_test'],0)]:
    i,l = load_images(folder,label)
    test_imgs+=i; test_lbls+=l

print(f"TRAIN: {sum(train_lbls)} anemic / {len(train_lbls)} total")
print(f"VAL: {sum(val_lbls)} anemic / {len(val_lbls)} total")
print(f"TEST: {sum(test_lbls)} anemic / {len(test_lbls)} total")

# =========================
# DATASET
# =========================
class SingleEyeDataset(Dataset):
    def __init__(self, imgs, labels, transform):
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.imgs)
    def __getitem__(self, idx):
        return self.transform(self.imgs[idx]), self.labels[idx]

def seed_worker(worker_id):
    np.random.seed(SEED + worker_id)
    random.seed(SEED + worker_id)
    torch.manual_seed(SEED + worker_id)

# =========================
# MODEL
# =========================
class SingleResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512,1)
    def forward(self,x):
        return self.backbone(x)

# =========================
# METRICS
# =========================
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    preds, probs, labels_all = [], [], []
    for imgs, labels in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device).float().unsqueeze(1)
        out = model(imgs)
        p = torch.sigmoid(out).cpu().numpy().flatten()
        preds.extend((p>0.5).astype(int))
        probs.extend(p)
        labels_all.extend(labels.cpu().numpy().flatten())
    if len(set(labels_all))<2:
        return float("nan"),float("nan"),float("nan"),float("nan"),float("nan"),0,0,0,0
    P,R,F1,_ = precision_recall_fscore_support(labels_all,preds,average='binary')
    acc = accuracy_score(labels_all,preds)
    auc = roc_auc_score(labels_all,probs)
    tn,fp,fn,tp = confusion_matrix(labels_all,preds,labels=[0,1]).ravel()
    return P,R,F1,acc,auc,tp,tn,fp,fn

# =========================
# TRAINING LOOP (FIXED: CV uses ONLY train data)
# =========================
def train_and_eval_single():
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((RESOLUTION,RESOLUTION)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    # 🔥 CRITICAL FIX: Use ONLY original training data for CV
    X = train_imgs  # ← Only training images
    y = train_lbls  # ← Only training labels
    
    if len(y) < N_SPLITS:
        raise RuntimeError("Not enough training samples for CV")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    results = []
    
    for fold, (tr_idx, vl_idx) in enumerate(kf.split(X, y), 1):
        print(f"\n--- Fold {fold} ---")
        
        # Create train/val splits from ORIGINAL TRAINING DATA ONLY
        train_subset_imgs = [X[i] for i in tr_idx]
        train_subset_lbls = [y[i] for i in tr_idx]
        val_subset_imgs = [X[i] for i in vl_idx]
        val_subset_lbls = [y[i] for i in vl_idx]
        
        tr_loader = DataLoader(
            SingleEyeDataset(train_subset_imgs, train_subset_lbls, train_tf),
            batch_size=BATCH_CV, shuffle=True, num_workers=NUM_WORKERS,
            worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
            generator=torch.Generator().manual_seed(SEED)
        )
        vl_loader = DataLoader(
            SingleEyeDataset(val_subset_imgs, val_subset_lbls, eval_tf),
            batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
        )

        model = SingleResNet18().to(device)
        opt = optim.Adam(model.parameters(), lr=LR_CV)
        loss_fn = nn.BCEWithLogitsLoss()
        scaler = GradScaler(enabled=USE_AMP and device.type == "cuda")

        for ep in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in tr_loader:
                imgs = imgs.to(device).float()
                labels = labels.to(device).float().unsqueeze(1)
                opt.zero_grad(set_to_none=True)
                with autocast(enabled=USE_AMP and device.type == "cuda"):
                    out = model(imgs)
                    loss = loss_fn(out, labels)
                scaler.scale(loss).backward()
                scaler.step(opt)
                scaler.update()
                total_loss += loss.item()
            
            # Print loss every 20 epochs
            if (ep + 1) % 20 == 0 or ep == EPOCHS_CV - 1:
                print(f"Epoch {ep+1}/{EPOCHS_CV} Loss: {total_loss:.4f}")
            
            # Early stopping check (every epoch for safety)
            if EARLY_STOP_PR:
                P, R, _, _, _, _, _, _, _ = evaluate(model, vl_loader)
                if P >= EARLY_STOP_PR and R >= EARLY_STOP_PR:
                    print(f"✅ Early stop at epoch {ep+1}: P={P:.3f}, R={R:.3f}")
                    break

        # Final evaluation on fold's validation set
        val_metrics = evaluate(model, vl_loader)
        results.append({
             'Fold': fold,
             'Val_Precision': val_metrics[0],
             'Val_Recall': val_metrics[1],
             'Val_F1': val_metrics[2],
             'Val_Accuracy': val_metrics[3],
             'Val_AUC': val_metrics[4],
             'Val_TP': val_metrics[5],
             'Val_TN': val_metrics[6],
             'Val_FP': val_metrics[7],
             'Val_FN': val_metrics[8],
         })
        print(f"Fold {fold} → P={val_metrics[0]:.3f}, R={val_metrics[1]:.3f}")
        
        if SAVE_EVERY_FOLD_MODEL:
             torch.save({
                 'model_state': model.state_dict(),
                 'fold': fold,
                 'val_metrics': {
                     'precision': val_metrics[0],
                     'recall': val_metrics[1],
                     'f1': val_metrics[2],
                     'accuracy': val_metrics[3],
                     'auc': val_metrics[4],
                     'tp': val_metrics[5],
                     'tn': val_metrics[6],
                     'fp': val_metrics[7],
                     'fn': val_metrics[8],
                 }
             }, os.path.join(OUTPUT_DIR, f"fold_{fold}.pt"))

    # Save CV results

    _cv_cols = ['Fold','Val_Precision','Val_Recall','Val_F1','Val_Accuracy','Val_AUC','Val_TP','Val_TN','Val_FP','Val_FN']
    pd.DataFrame(results)[_cv_cols].to_csv(os.path.join(OUTPUT_DIR, "cv_results.csv"), index=False)
    # Select best fold
    df = pd.DataFrame(results)
    df['minPR'] = df[['Val_Precision', 'Val_Recall']].min(axis=1)
    candidates = df[(df.Val_Precision >= 0.90) & (df.Val_Recall >= 0.90)]
    
    if len(candidates) > 0:
        best = candidates.sort_values(['Val_F1', 'Val_AUC', 'minPR'], ascending=False).iloc[0]
    else:
        best = df.sort_values(['minPR', 'Val_F1', 'Val_AUC'], ascending=False).iloc[0]
    
    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | P={best['Val_Precision']:.3f}, R={best['Val_Recall']:.3f}")

    # =========================
    # FINAL EVALUATION ON ORIGINAL TEST SET
    # =========================


    # =========================
    # FINAL EVALUATION ON ORIGINAL TEST SET
    # =========================
    test_loader = DataLoader(
        SingleEyeDataset(test_imgs, test_lbls, eval_tf),
        batch_size=BATCH_CV, shuffle=False, num_workers=NUM_WORKERS
    )

    # Load best model (fix PyTorch 2.6 loading)
    checkpoint = torch.load(
        os.path.join(OUTPUT_DIR, f"fold_{best_fold}.pt"),
        map_location=device,
        weights_only=False  # <--- add this if using PyTorch >=2.6
    )
    model = SingleResNet18().to(device)
    model.load_state_dict(checkpoint['model_state'])

    # Evaluate on ORIGINAL TEST SET
    test_metrics = evaluate(model, test_loader)

    print("\n📊 FINAL TEST RESULTS (Original Test Set):")
    print(f"Precision: {test_metrics[0]:.4f}")
    print(f"Recall:    {test_metrics[1]:.4f}")
    print(f"F1 score:  {test_metrics[2]:.4f}")
    print(f"Accuracy:  {test_metrics[3]:.4f}")
    print(f"AUC:       {test_metrics[4]:.4f}")
    print(f"TP, TN, FP, FN: {int(test_metrics[5])}, {int(test_metrics[6])}, {int(test_metrics[7])}, {int(test_metrics[8])}")
  
    _test_row = [{
         'Test_Precision': test_metrics[0],
         'Test_Recall': test_metrics[1],
         'Test_F1': test_metrics[2],
         'Test_Accuracy': test_metrics[3],
         'Test_AUC': test_metrics[4],
         'Test_TP': test_metrics[5],
         'Test_TN': test_metrics[6],
         'Test_FP': test_metrics[7],
         'Test_FN': test_metrics[8],
         'Best_Fold': best_fold
    }]
    _test_cols = ['Test_Precision','Test_Recall','Test_F1','Test_Accuracy','Test_AUC','Test_TP','Test_TN','Test_FP','Test_FN','Best_Fold']
    pd.DataFrame(_test_row)[_test_cols].to_csv(os.path.join(OUTPUT_DIR, "test_results.csv"), index=False)
    return model
'''
# =========================
# RUN
# =========================
if __name__ == "__main__":
    train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
'''
''' single right eye 3 alone model results'''

# =========================
# TFLITE CONVERSION
# =========================

def convert_to_tflite(model, output_dir, resolution):
    """
    Converts a trained PyTorch model to TFLite format via ONNX and TensorFlow.
    """
    import torch.onnx
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    
    # Suppress ONNX-TF/TF warnings
    warnings.filterwarnings("ignore", category=FutureWarning)
    
    # --- 1. Preparation and Tracing ---
    model.eval()
    dummy_input = torch.randn(1, 3, resolution, resolution, device=device)
    onnx_path = os.path.join(output_dir, "model.onnx")
    tf_path = os.path.join(output_dir, "tf_model")
    tflite_path = os.path.join(output_dir, "single_eye_resnet18.tflite")
    
    print("\n--- Starting TFLite Conversion Pipeline ---")
    
    # --- 2. PyTorch to ONNX ---
    print("1. Converting PyTorch model to ONNX...")
    try:
        # Export the model
        torch.onnx.export(
            model,
            dummy_input,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input'],
            output_names=['output'],
            dynamic_axes={'input': {0: 'batch_size'}}
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
        
    except Exception as e:
        print(f"   ❌ PyTorch to ONNX failed: {e}")
        return

    # --- 3. ONNX to TensorFlow SavedModel ---
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        # Load the ONNX model
        onnx_model = onnx.load(onnx_path)
        
        # Prepare the TensorFlow backend
        tf_rep = prepare(onnx_model)
        
        # Export the model to TensorFlow SavedModel format
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
        
    except Exception as e:
        print(f"   ❌ ONNX to TensorFlow SavedModel failed. Make sure onnx-tf and tensorflow are installed. Error: {e}")
        return

    # --- 4. TensorFlow SavedModel to TFLite ---
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        # Initialize the TFLite converter
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        
        # Convert the model
        tflite_model = converter.convert()
        
        # Save the TFLite model
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)
        
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   TFLite Model Size: {os.path.getsize(tflite_path) / (1024*1024):.2f} MB")
        
    except Exception as e:
        print(f"   ❌ TensorFlow to TFLite failed: {e}")
        return

# =========================
# RUN (Update)
# =========================
if __name__ == "__main__":
    # The trained model 'zz' is returned by train_and_eval_single()
    zz = train_and_eval_single()
    print("\n✅ Single-eye model finished reproducibly with NO DATA LEAKAGE.")
    
    # ⭐ ADDED STEP: Convert to TFLite
    convert_to_tflite(zz, OUTPUT_DIR, RESOLUTION)
    print("✅ TFLite conversion pipeline complete.")

    

In [1]:
# -*- coding: utf-8 -*-
"""
Reproducible Tri-right-Eye ResNet18 (5-fold CV -> pick best fold -> TEST)
------------------------------------------------------------------------
Determinism safeguards:
- Fixed global seed (random, numpy, torch, CUDA)
- cuDNN deterministic + disable benchmarking
- torch.use_deterministic_algorithms(True)
- Optional: export PYTHONHASHSEED=42 before launching Python
- OpenCV single-threaded I/O
- Sorted directory listings and tri-eye base-ID intersections
- StratifiedKFold with fixed random_state
- DataLoader generator seeded; NUM_WORKERS = 0 (strict determinism)
- Deterministic transforms (no randomness beyond our fixed Python RNG)
"""

import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

# =========================
# GLOBAL CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0          # keep 0 for strict reproducibility
PIN_MEMORY = False       # can be True on CUDA; doesn't affect determinism
USE_AMP = True           # AMP is deterministic under fixed seeds + deterministic algos
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTIONS = [224]
EPOCHS_CV = 120
BATCH_CV = 22
LR_CV = 0.00022

'''
resolutions = [224]
EPOCHS_CV = 120
EPOCHS_FINAL = EPOCHS_CV
BATCH_CV = 22
LR_CV = 0.00022
'''

# =========================
# DEVICE + DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # ban nondeterministic kernels; warn_only True to avoid hard-crash if a path isn't supported
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        # keep matmul & TF32 paths consistent
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)  # single-threaded OpenCV
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS (edit base_path if needed)
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "tri_tongue_hb_90_repro_bestfold_only")
os.makedirs(output_dir, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

train_dirs_anemic = make_full_path({
    'right1': 'tri_tongue/tongue_1_hb_less_than_9_0/tongue_extracted/anemic_train_roi/',
    'right2': 'tri_tongue/tongue_2_hb_less_than_9_0/tongue_extracted/anemic_train_roi/',
    'right3': 'tri_tongue/tongue_3_hb_less_than_9_0/tongue_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'right1': 'tri_tongue/tongue_1_hb_less_than_9_0/tongue_extracted/anemic_not_train_roi/',
    'right2': 'tri_tongue/tongue_2_hb_less_than_9_0/tongue_extracted/anemic_not_train_roi/',
    'right3': 'tri_tongue/tongue_3_hb_less_than_9_0/tongue_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'right1': 'tri_tongue/tongue_1_hb_less_than_9_0/tongue_extracted/anemic_val_roi/',
    'right2': 'tri_tongue/tongue_2_hb_less_than_9_0/tongue_extracted/anemic_val_roi/',
    'right3': 'tri_tongue/tongue_3_hb_less_than_9_0/tongue_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'right1': 'tri_tongue/tongue_1_hb_less_than_9_0/tongue_extracted/anemic_not_val_roi/',
    'right2': 'tri_tongue/tongue_2_hb_less_than_9_0/tongue_extracted/anemic_not_val_roi/',
    'right3': 'tri_tongue/tongue_3_hb_less_than_9_0/tongue_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'right1': 'tri_tongue/tongue_1_hb_less_than_9_0/tongue_extracted/anemic_test_roi/',
    'right2': 'tri_tongue/tongue_2_hb_less_than_9_0/tongue_extracted/anemic_test_roi/',
    'right3': 'tri_tongue/tongue_3_hb_less_than_9_0/tongue_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'right1': 'tri_tongue/tongue_1_hb_less_than_9_0/tongue_extracted/anemic_not_test_roi/',
    'right2': 'tri_tongue/tongue_2_hb_less_than_9_0/tongue_extracted/anemic_not_test_roi/',
    'right3': 'tri_tongue/tongue_3_hb_less_than_9_0/tongue_extracted/anemic_not_test_roi/'
})
# =========================
# UTIL (tri-right intersection, loading, logging)
# =========================
def base_from(fname, suffix):
    if not fname.endswith(suffix):
        return None
    return fname[: -len(suffix)]

def common_bases_right(dirs_map):
    """
    Intersection of base_ids for right1-right2-right3 (sorted).
    """
    suffixes = {
        'right1': '_tongue_1.png',
        'right2': '_tongue_2.png',
        'right3': '_tongue_3.png',
    }
    bases_sets = []
    for k in ['right1', 'right2', 'right3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            print(f"⚠️ Folder missing: {folder}")
            return []
        names = sorted([f for f in os.listdir(folder) if f.endswith(suffixes[k])])
        bases = set()
        for f in names:
            b = base_from(f, suffixes[k])
            if b:
                bases.add(b)
        bases_sets.append(bases)
    common = set.intersection(*bases_sets) if bases_sets else set()
    return sorted(list(common))

def load_tri_images_by_bases(dirs_map, bases):
    """
    Load the three right-eye images per base (RGB).
    """
    out = {'r1': [], 'r2': [], 'r3': []}
    key_map = {
        'r1': ('right1', '_tongue_1.png'),
        'r2': ('right2', '_tongue_2.png'),
        'r3': ('right3', '_tongue_3.png'),
    }
    ok = 0
    for b in bases:
        imgs = {}
        failed = False
        for short_k, (long_k, suf) in key_map.items():
            path = os.path.join(dirs_map[long_k], b + suf)
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[short_k] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if failed:
            continue
        for k in out.keys():
            out[k].append(imgs[k])
        ok += 1
    return out, ok

def prepare_dataset_right(anemic_dirs, non_dirs, split_name="(split)"):
    """
    Enforce strict tri-right-eye match per class by intersecting base ids across the three right-eye folders.
    """
    bases_anemic = common_bases_right(anemic_dirs)
    bases_non = common_bases_right(non_dirs)

    print(f"\n🔎 {split_name} (right) - discovered bases: anemic={len(bases_anemic)}, non-anemic={len(bases_non)}")

    imgs_a, ca = load_tri_images_by_bases(anemic_dirs, bases_anemic)
    imgs_n, cn = load_tri_images_by_bases(non_dirs,  bases_non)

    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1'])
    }

    print(f"✅ {split_name}: tri-matched right samples -> anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")

    # Save used base IDs (audit reproducibility)
    used_bases_df = pd.DataFrame({
        'class': (['anemic']*len(bases_anemic)) + (['non_anemic']*len(bases_non)),
        'base_id': bases_anemic + bases_non
    })
    used_bases_df.to_csv(os.path.join(output_dir, f"{split_name.lower()}_used_base_ids_right.csv"), index=False)

    return data

def count_files(d):
    return sum(1 for f in sorted(os.listdir(d)) if f.endswith(".png")) if os.path.isdir(d) else 0

def print_dir_stats(title, dirs_map):
    print(f"\n📂 {title}")
    for k in ['right1','right2','right3']:
        p = dirs_map[k]
        c = count_files(p)
        print(f"{k:7s} | {p} | files={c}")

# =========================
# BEFORE MODEL: print raw file counts per dir
# =========================
print_dir_stats("TRAIN anemic (right)", train_dirs_anemic)
print_dir_stats("TRAIN non-anemic (right)", train_dirs_non)
print_dir_stats("VAL   anemic (right)", val_dirs_anemic)
print_dir_stats("VAL   non-anemic (right)", val_dirs_non)
print_dir_stats("TEST  anemic (right)", test_dirs_anemic)
print_dir_stats("TEST  non-anemic (right)", test_dirs_non)

# =========================
# BUILD STRICT TRI-right DATASETS (REPRODUCIBLE)
# =========================
train_data = prepare_dataset_right(train_dirs_anemic, train_dirs_non, split_name="TRAIN")
val_data   = prepare_dataset_right(val_dirs_anemic,   val_dirs_non,   split_name="VAL")
test_data  = prepare_dataset_right(test_dirs_anemic,  test_dirs_non,  split_name="TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No tri-right-eye TRAIN samples found. Check paths/filenames.")
if len(val_data['label']) == 0:
    print("⚠️ No tri-right-eye VAL samples found. We'll still run TEST eval for best fold if available.")
if len(test_data['label']) == 0:
    print("⚠️ No tri-right-eye TEST samples found. Final test evaluation will be skipped.")

# =========================
# DATASET / DATALOADER
# =========================
class TrirightDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data['label'])

    def __getitem__(self, idx):
        images = [self.data[k][idx] for k in ['r1','r2','r3']]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        return images, label

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type=='cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
        persistent_workers=False
    )

# =========================
# MODEL (3x ResNet18 + MLP head)
# =========================
class TriResNetright(nn.Module):
    def __init__(self):
        super().__init__()
        def res():
            m = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
            m.fc = nn.Identity()
            return m
        self.models = nn.ModuleList([res() for _ in range(3)])
        self.fc = nn.Sequential(
            nn.Linear(3*512, 512),
            nn.LayerNorm(512),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(512, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(0.25),
            nn.Linear(128, 1)
        )

    def forward(self, *x):
        feats = [model(xi) for model, xi in zip(self.models, x)]
        x = torch.cat(feats, dim=1)
        return self.fc(x)

# =========================
# EVALUATION
# =========================
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    for imgs, labels in loader:
        imgs = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        out = model(*imgs)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(all_labels)) < 2:
        p = r = f1 = auc = float('nan')
        acc = accuracy_score(all_labels, all_preds)
        tn = fp = fn = tp = 0
        try:
            tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
        except Exception:
            pass
        return p, r, f1, acc, auc, tp, tn, fp, fn

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn
# -*- coding: utf-8 -*-
"""
Reproducible Tri-tongue-Eye ResNet18 (5-fold CV -> pick best fold -> TEST -> TFLite)
---------------------------------------------------------------------
Fully deterministic + converts best fold model to ONNX → TensorFlow → TFLite.
"""

# (All your imports, global config, dataset code, and training sections remain unchanged)

# =========================
# ✅ TFLITE CONVERSION FUNCTION (fixed)
# =========================
def convert_to_tflite(best_model: nn.Module, output_dir: str, resolution: int):
    """
    Converts the tri-tongue-eye PyTorch model to TFLite via ONNX and TensorFlow.
    Input: three (B,3,H,W) tensors → Output: (B,1)
    """
    import os
    import torch
    import onnx
    from onnx_tf.backend import prepare
    import tensorflow as tf
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    os.makedirs(output_dir, exist_ok=True)

    onnx_path = os.path.join(output_dir, "tri_tongue_model.onnx")
    tf_path = os.path.join(output_dir, "tri_tongue_tf_model")
    tflite_path = os.path.join(output_dir, "tri_tongue_eye_resnet18.tflite")

    print("\n--- Starting TFLite Conversion Pipeline ---")

    # Step 1️⃣ PyTorch → ONNX
    print("1. Converting PyTorch model to ONNX...")
    best_model.eval().to(device)
    dummy_inputs = tuple(torch.randn(1, 3, resolution, resolution, device=device) for _ in range(3))

    try:
        torch.onnx.export(
            best_model,
            dummy_inputs,
            onnx_path,
            export_params=True,
            opset_version=13,
            do_constant_folding=True,
            input_names=['input1', 'input2', 'input3'],
            output_names=['output'],
            dynamic_axes={'input1': {0: 'batch'},
                          'input2': {0: 'batch'},
                          'input3': {0: 'batch'},
                          'output': {0: 'batch'}}
        )
        print(f"   ✅ ONNX model saved to: {onnx_path}")
    except Exception as e:
        print(f"   ❌ PyTorch → ONNX failed: {e}")
        return

    # Step 2️⃣ ONNX → TensorFlow
    print("2. Converting ONNX model to TensorFlow SavedModel...")
    try:
        onnx_model = onnx.load(onnx_path)
        tf_rep = prepare(onnx_model)
        tf_rep.export_graph(tf_path)
        print(f"   ✅ TensorFlow SavedModel saved to: {tf_path}")
    except Exception as e:
        print(f"   ❌ ONNX → TensorFlow conversion failed: {e}")
        return

    # Step 3️⃣ TensorFlow → TFLite
    print("3. Converting TensorFlow SavedModel to TFLite...")
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
        # Optional: quantization for smaller model size
        # converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()
        with open(tflite_path, "wb") as f:
            f.write(tflite_model)
        print(f"   ✅ TFLite model saved to: {tflite_path}")
        print(f"   📦 Size: {os.path.getsize(tflite_path)/(1024*1024):.2f} MB")
    except Exception as e:
        print(f"   ❌ TensorFlow → TFLite failed: {e}")
        return




# =========================
# TRAIN / VALIDATE via 5-fold CV (on TRAIN only)
# =========================
results = []

for resolution in RESOLUTIONS:
    print(f"\n===== Processing right resolution: {resolution} =====")
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    if len(train_data['label']) < N_SPLITS:
        raise RuntimeError(f"Too few right train samples for {N_SPLITS}-fold CV.")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    fold = 1
    labels_np = np.array(train_data['label'])

    cv_index_records = []

    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- right Fold {fold} ---")

        cv_index_records.append({
            "fold": fold,
            "train_indices": train_idx.tolist(),
            "val_indices": val_idx.tolist()
        })

        train_subset = {k: [v[i] for i in train_idx] for k, v in train_data.items()}
        val_subset   = {k: [v[i] for i in val_idx]   for k, v in train_data.items()}

        train_loader = make_loader(TrirightDataset(train_subset, train_transform), batch_size=BATCH_CV, shuffle=True)
        val_loader   = make_loader(TrirightDataset(val_subset,   test_transform),  batch_size=BATCH_CV, shuffle=False)

        model = TriResNetright().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels in train_loader:
                imgs = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(*imgs)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()
            print(f"Epoch [{epoch+1}/{EPOCHS_CV}] 🔁 Loss: {total_loss:.6f}")

        # fold VAL metrics
        val_metrics = evaluate(model, val_loader)
        result_row = {
            'EyeSet': 'right',
            'Resolution': resolution,
            'Fold': fold,
            'Epochs': EPOCHS_CV,
            'BatchSize': BATCH_CV,
            'LearningRate': LR_CV,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8]
        }
        results.append(result_row)
        print(result_row)

        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"right_cv_fold_{fold}_res{resolution}.pt")
            torch.save({'model_state': model.state_dict(),
                        'seed': SEED,
                        'resolution': resolution}, fold_path)

        fold += 1

    # save CV tables/indices
    if results:
        results_df = pd.DataFrame(results)
        results_df.to_csv(os.path.join(output_dir, f"{resolution}_right_val_cross_validation_results.csv"), index=False)

    pd.DataFrame(cv_index_records).to_json(os.path.join(output_dir, f"{resolution}_right_cv_indices.json"),
                                           orient='records', indent=2)

    # =========================
    # SELECT BEST FOLD & EVALUATE ON TEST (no retraining)
    # =========================
    print("\n🔎 Selecting best fold by validation metrics (require Val P & R ≥ 0.90 if available)...")
    results_df = pd.DataFrame(results)
    results_df['min_PR'] = results_df[['Val_Precision','Val_Recall']].min(axis=1)

    # candidates meeting target
    candidates = results_df[(results_df['Val_Precision'] >= 0.90) &
                            (results_df['Val_Recall']    >= 0.90)]

    if len(candidates) > 0:
        # strongest among those meeting the threshold
        best = candidates.sort_values(['Val_F1','Val_AUC','min_PR'], ascending=False).iloc[0]
    else:
        # fallback: best compromise (highest min(PR), then F1, then AUC)
        best = results_df.sort_values(['min_PR','Val_F1','Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold} | "
          f"Val_P={best['Val_Precision']:.3f}, Val_R={best['Val_Recall']:.3f}, "
          f"Val_F1={best['Val_F1']:.3f}, Val_AUC={best['Val_AUC']:.3f}")

    # persist selection
    with open(os.path.join(output_dir, f"{resolution}_right_best_fold.txt"), "w") as f:
        f.write(str(best_fold))

    # load its checkpoint
    ckpt_path = os.path.join(output_dir, f"right_cv_fold_{best_fold}_res{resolution}.pt")
    state = torch.load(ckpt_path, map_location=device)
    best_model = TriResNetright().to(device)
    best_model.load_state_dict(state['model_state'])

    # build loaders for VAL (optional audit) and TEST with identical batch size
    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Optional: evaluate chosen model on the *held-out* configured VAL split too (for auditing)
    if len(val_data['label']) > 0:
        chosen_val_loader = make_loader(TrirightDataset(val_data, test_transform),
                                        batch_size=BATCH_CV, shuffle=False)
        chosen_val_metrics = evaluate(best_model, chosen_val_loader)
        print("\n📊 VALIDATION (best-fold model) Results")
        print(f"Precision: {chosen_val_metrics[0]:.4f}, Recall: {chosen_val_metrics[1]:.4f}, F1: {chosen_val_metrics[2]:.4f}")
        print(f"Accuracy:  {chosen_val_metrics[3]:.4f}, AUC: {chosen_val_metrics[4]:.4f}")
        pd.DataFrame([{
            'ChosenFold': best_fold,
            'Val_Precision': chosen_val_metrics[0], 'Val_Recall': chosen_val_metrics[1], 'Val_F1': chosen_val_metrics[2],
            'Val_Accuracy': chosen_val_metrics[3], 'Val_AUC': chosen_val_metrics[4], 'Val_TP': chosen_val_metrics[5],
            'Val_TN': chosen_val_metrics[6], 'Val_FP': chosen_val_metrics[7], 'Val_FN': chosen_val_metrics[8]
        }]).to_csv(os.path.join(output_dir, f"{resolution}_right_bestfold_val_results.csv"), index=False)

    # TEST evaluation with the best-fold model
    if len(test_data['label']) > 0:
        test_loader = make_loader(TrirightDataset(test_data, test_transform),
                                  batch_size=BATCH_CV, shuffle=False)
        test_metrics = evaluate(best_model, test_loader)
        print("\n📊 TEST (best-fold model) Results")
        print(f"Precision: {test_metrics[0]:.4f}, Recall: {test_metrics[1]:.4f}, F1: {test_metrics[2]:.4f}")
        print(f"Accuracy:  {test_metrics[3]:.4f}, AUC: {test_metrics[4]:.4f}")

        pd.DataFrame([{
            'ChosenFold': best_fold,
            'Test_Precision': test_metrics[0], 'Test_Recall': test_metrics[1], 'Test_F1': test_metrics[2],
            'Test_Accuracy': test_metrics[3], 'Test_AUC': test_metrics[4], 'Test_TP': test_metrics[5],
            'Test_TN': test_metrics[6], 'Test_FP': test_metrics[7], 'Test_FN': test_metrics[8]
        }]).to_csv(os.path.join(output_dir, f"{resolution}_right_bestfold_test_results.csv"), index=False)
    print("\n✅ tongue: Done. All discovery, 5-fold CV, model selection, and TEST evaluation are reproducible.")
    print("\n🚀 Starting final TFLite export for Tri tongue Eye model...")
    try:
        convert_to_tflite(best_model, output_dir, resolution)
        print("✅ Tri tongue Eye TFLite conversion pipeline completed successfully.")
    except Exception as e:
        print(f"❌ Conversion pipeline failed: {e}")


print("\n✅ right: Done. All discovery, 5-fold CV, model selection, and TEST evaluation are reproducible.")


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using device: cuda
GPU: NVIDIA H100 80GB HBM3

📂 TRAIN anemic (right)
right1  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue/tongue_1_hb_less_than_9_0/tongue_extracted/anemic_train_roi/ | files=261
right2  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue/tongue_2_hb_less_than_9_0/tongue_extracted/anemic_train_roi/ | files=259
right3  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue/tongue_3_hb_less_than_9_0/tongue_extracted/anemic_train_roi/ | files=253

📂 TRAIN non-anemic (right)
right1  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue/tongue_1_hb_less_than_9_0/tongue_extracted/anemic_not_train_roi/ | files=1953
right2  | /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue/tongue_2_hb_less_than_9_0/tongue_extracted/anemic_not_train_roi/ | files=1945
right3  | /home/u

2025-10-14 07:25:30.332678: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-14 07:25:30.334254: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-14 07:25:30.368451: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-14 07:25:30.949267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



--- Starting TFLite Conversion Pipeline ---
1. Converting PyTorch model to ONNX...
   ✅ ONNX model saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue_hb_90_repro_bestfold_only/tri_tongue_model.onnx
2. Converting ONNX model to TensorFlow SavedModel...


2025-10-14 07:25:34.407536: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-14 07:25:34.409164: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_187_x, onnx_tf_prefix__fc_fc_1_add_1_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1

INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue_hb_90_repro_bestfold_only/tri_tongue_tf_model/assets


INFO:tensorflow:Assets written to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue_hb_90_repro_bestfold_only/tri_tongue_tf_model/assets
INFO:absl:Writing fingerprint to /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue_hb_90_repro_bestfold_only/tri_tongue_tf_model/fingerprint.pb


   ✅ TensorFlow SavedModel saved to: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue_hb_90_repro_bestfold_only/tri_tongue_tf_model
3. Converting TensorFlow SavedModel to TFLite...
   ❌ TensorFlow → TFLite failed: <unknown>:0: error: loc(callsite(callsite(fused["Erf:", "onnx_tf_prefix_/fc/fc.2/Erf@__inference___call___1631"] at fused["PartitionedCall:", "PartitionedCall@__inference_signature_wrapper_1899"]) at fused["PartitionedCall:", "PartitionedCall"])): 'tf.Erf' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["PartitionedCall:", "PartitionedCall"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["Erf:", "onnx_tf_prefix_/fc/fc.2/Erf@__inference___call___1631"] at fused["PartitionedCall:", "PartitionedCall@__inference_signature_wrapper_1899"]) at fused["PartitionedCall:", "PartitionedCall"])): Error code: ERROR_NEEDS_FLEX_OPS
<unknown>:0: error: loc(callsite(callsite(fused["Erf:", "onnx_tf_prefix_/fc/fc.6/Erf

2025-10-14 07:25:47.686311: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-10-14 07:25:47.686358: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-10-14 07:25:47.688468: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue_hb_90_repro_bestfold_only/tri_tongue_tf_model
2025-10-14 07:25:47.800526: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-10-14 07:25:47.800567: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/tri_tongue_hb_90_repro_bestfold_only/tri_tongue_tf_model
2025-10-14 07:25:47.914854: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-10-1